In [2]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from typing import TypedDict
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
class BlogState(TypedDict):
    topic:str
    outline:str
    blog:str
    feedback:str
    rating:str

In [4]:
def load_llm():
    api_key=os.getenv('GEMINI_KEY')
    llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key=api_key)
    return llm

In [7]:
def outline_generation(state:BlogState)->BlogState:
    llm=load_llm()
    topic=state["topic"]
    template=''' You are a powerful AI Assistant, Create a proper outline of a following {topic} with respect to blog generation'''
    prompt=PromptTemplate(input_variables=["topic"],template=template)
    chain=prompt|llm
    result=chain.invoke({"topic":topic})
    state["outline"]=result.content
    return state


def blog_generation(state:BlogState)->BlogState:
    llm=load_llm()
    topic=state["topic"]
    outline=state["outline"]
    template=''' You are an Powerful AI assistant, Generate a informative blog on a topic:{topic} with a proper outline:{outline}'''
    prompt=PromptTemplate(input_variables=["topic","outline"],template=template)

    chain=prompt|llm
    result=chain.invoke({"topic":topic,"outline":outline})
    state["blog"]=result.content
    return state

def feedback_generation(state:BlogState):
    llm=load_llm()
    blog=state["blog"]
    template=''' You are an Powerful AI assistant, Generate a small feedback on the following blog:{blog}'''
    prompt=PromptTemplate(input_variables=["blog"],template=template)

    chain=prompt|llm
    result=chain.invoke({"blog":blog})
    feedback=result.content
    return {"feedback":feedback}

def rating_generation(state:BlogState):
    llm=load_llm()
    blog=state["blog"]
    template=''' You are an Powerful AI assistant,rate the following blog:{blog} out of 10'''
    prompt=PromptTemplate(input_variables=["blog"],template=template)

    chain=prompt|llm
    result=chain.invoke({"blog":blog})
    rating=result.content
    return {"rating":rating}


In [9]:
graph=StateGraph(BlogState)

# adding nodes
graph.add_node("outline generation",outline_generation)
graph.add_node("blog generation",blog_generation)
graph.add_node("feedback_generation",feedback_generation)
graph.add_node("rating_generation",rating_generation)

# adding edges
graph.add_edge(START,"outline generation")
graph.add_edge("outline generation","blog generation")
graph.add_edge("blog generation","feedback_generation")
graph.add_edge("blog generation","rating_generation")

graph.add_edge("feedback_generation",END)
graph.add_edge("rating_generation",END)

#compiling edges
workflow=graph.compile()

In [10]:
result=workflow.invoke({"topic":"Cuthulu"})

In [11]:
print(result["rating"])
print(result["feedback"])

This blog post is absolutely **phenomenal**. It's a masterclass in informative, engaging, and well-structured writing.

**Rating: 10/10**

Here's a breakdown of why it earns a perfect score:

**Strengths:**

1.  **Impeccable Structure and Flow:** The outline is followed perfectly, with clear, logical transitions between sections. Each heading promises a specific topic, and the content delivers precisely.
2.  **Engaging Introduction:** It hooks the reader immediately with thought-provoking questions and clearly sets the stage for what's to come, defining cosmic horror early on.
3.  **Depth of Information:** You've covered all the essential aspects of Cthulhu and Lovecraft:
    *   Lovecraft's biography and philosophical worldview (Cosmicism).
    *   The genesis of "The Call of Cthulhu" and the Mythos.
    *   A vivid, accurate description of Cthulhu's appearance and nature (Great Old One, R'lyeh, slumber, mental corruption).
    *   The profound philosophical dread it embodies (existen